### JavaScript Execution
JavaScript specification doesn't specify if JavaScript has to be compiled or interpreted. JavaScript infact runs in two phases. In the first phase the code is compiled/parsed into executable instructions and in the second it actually executes. This two phase execution is evident from the following code snippets:

In [ ]:
// None of the below code will execute
// Instead we'll get an error:
// Uncaught SyntaxError: expected expression, got '.'
var greeting = "Hello";
console.log(greeting);
greeting = ."Hi";

Another example:

In [ ]:
// The below console log will fail
console.log("Hey");

saySomething("Hello","Hi");
// Uncaught SyntaxError: Duplicate parameter name not
// allowed in this context

function saySomething(greeting,greeting) {
    "use strict";
    console.log(greeting);
}

The two examples above show that JavaScript first goes through the entire code and then throws error if it finds any. Rather than executing code line by line and then throwing errors.

### Scope
JavaScript decides the scope of all variables involved at compile/parse time. This scope is referred to as *lexical scope*.

In non-script mode, it is possible to modify the scope of variable during run-time. This is achieved using `eval` or using `with` statement. For example:

In [ ]:
var myVariable = 'From global';

function someFunction(){
    eval("var myVariable = 'Using eval';");
    console.log(myVariable);
}

someFunction(); // Using eval

The `var` keyword understands global scope and function scope. Whereas `let` and `const` keywords are block scoped.  

In the first phase of execution, the JavaScript assigns variable name to scope whenever it sees the `var` keyword. However during execution if it finds an assignment statement involving a variable name not encountered earlier, it assigns the variable to global scope.

In [ ]:
function smallTest(){
    thisWillBeGlobal = 25; // This variable was not identified during first phase
}

smallTest()
console.log(window.thisWillBeGlobal); // window is the global object in case of
                                      // JavaScript

**Shadowing** variable in inner scope shadows variable in outer scope having the same name.

In [6]:
var studentName = "Mongo";

function printStudent(studentName) { // studentName parameter shadows the global
                                    // studentName variable
    studentName = studentName.toUpperCase();
    console.log(studentName);
}

printStudent('Maximus');
console.log(studentName);

MAXIMUS
Mongo


One thing to note is that `let` can shadow `var`, but the reverse is not possible.

In [8]:
function something() {
    var special = "JavaScript";

    {
        let special = 42;   // totally fine shadowing

        // ..
    }
}

function another() {
    // ..

    {
        let special = "JavaScript";

        {
            var special = "JavaScript";
            // ^^^ Syntax Error

            // ..
        }
    }
}

18:17 - Cannot initialize outer scoped variable 'special' in the same scope as block scoped declaration 'special'.


**Function Name Scope**

In [ ]:
// We can define a function in the following way:

var greet = function say(msg){ // We can assume that the scope for say function is
                              // the greet function scope
    console.log(msg);
}

greet('Welcome')
console.log(say) // Uncaught ReferenceError: say is not defined

**Global Scope and Let Keyword** the `let` keyword doesn't assign to global scope

In [ ]:
window.aNumber = 254;

let aNumber = "Twenty One";

console.log(aNumber); // Twenty One

console.log(window.aNumber);// 254

**Redeclaration Within Scope** If a declaration of a variable has been done using `var`, redeclaration of the same variable is just ignored.

In [14]:
var civilization = "Aztecs";
console.log(civilization); // Aztecs

var civilization; // Ignored
console.log(civilization); // Aztecs

Aztecs
Aztecs


Redeclaration using `let` however is a `SyntaxError`. Mixing `let` and `var` will also throw error.

**Scope and Try Catch Block** the catch block creates its own scope, thus the error variable is accessible only within that scope.

In [16]:
try{
    unknownFunction();
} catch(err) {
    console.log(err)
}

// err is not accessible here
console.log(err) // ReferenceError

2:5 - Cannot find name 'unknownFunction'.
8:13 - Cannot find name 'err'.


**Function Declaration In Block** The JavaScript specification says that function declarations are block scoped. However not all environements (browsers and even Node) stick to this specification because block scope was defined only in ES6.

In [ ]:
if(true){
    function blockScopedFunction(){
        console.log('A block scoped function')
    }
}

blockScopedFunction() // 'A block scoped function'

What about the following case?

In [ ]:
if(false){
    function blockScopedFunction(){
        console.log('A block scoped function')
    }
}

blockScopedFunction() // TypeError

In [18]:
if(true){
    function callMe(){
        console.log('First')
    }
}

if(true){
    function callMe(){
        console.log('Second')
    }
}

callMe()

function callMe(){
    console.log('Third')
}

Second


### Hoisting
Variable being visible from the beginning of its enclosing scope, even though its declaration may appear further down in the scope, is called *hoisting*. In the below example, a case of function hoisting, two things occur:
1. The identifier *hoistedFunction* is available from the beginning
2. The identifier has reference to the function attached to it, so that we can call it

In [11]:
hoistedFunction();

function hoistedFunction(){
    console.log('Function call');
}

Function call


Function hoisting only applies to formal function declarations, not to function expression assignments ie. hoisting applies to function declaration, not to function expression.

In [12]:
greeting();
// TypeError

var greeting = function greeting() {
    console.log("Hello!");
};

evalmachine.<anonymous>:2
greeting();
^

TypeError: greeting is not a function
    at evalmachine.<anonymous>:2:1
    at evalmachine.<anonymous>:9:3
    at sigintHandlersWrap (vm.js:288:15)
    at Script.runInContext (vm.js:130:14)
    at Object.runInContext (vm.js:311:6)
    at Object.execute (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\executor.js:159:38)
    at JupyterHandlerImpl.handleExecuteImpl (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\jupyter.js:206:38)
    at count.execQueue.add (C:\Users\salma\AppData\Roaming\npm\node_modules\tslab\dist\jupyter.js:164:57)


Notice, that we get `TypeError`, not `ReferenceError`. This means that JavaScript identifies the identifier greeting. But it doesn't know that it is a function.  

Similar to function hoisting, we have *variable hoisting*.

In [13]:
console.log(message)

var message = 'A message...'

undefined


When a variable is hoisted, the identifier is available from the beginning and the variable is assigned value undefined.

### Closure
*Closure* as a topic is only relevant to functions. The effect of closure can be illustrated by the following example:

In [21]:
function recipeGenerator(recipeId){
    var recipes = [
        {id: 1, ingredients: ['lemon', 'water', 'sugar', 'ice']},
        {id: 2, ingredients: ['orange', 'salt', 'sugar']},
        {id: 3, ingredients: ['chicken', 'salt', 'olive oil']}
    ]
    
    return function printRecipe(){
        var recipe = recipes.find(recipe => recipe.id == recipeId)
        
        console.log(`Recipe for id ${recipe.id}`)
        for(let i of recipe.ingredients){
            console.log(i)
        }
    }
}

var lemonadeRecipe = recipeGenerator(1)
lemonadeRecipe()

var orangeJuiceRecipe = recipeGenerator(2)
orangeJuiceRecipe()

Recipe for id 1
lemon
water
sugar
ice
Recipe for id 2
orange
salt
sugar


If there was no closure, the variables recipeId and recipes should have been garbage collected once the call to `recipeGenerator` was finished. But it is not the case here. All variables in `recipeGenerator` scope are preserved each execution.  

Closure allows `printRecipe` to continue to access those outer variables even after the outer scope is finished.

In [23]:
function callCounter(){
    var counter = 0;
    
    return function(){
        counter++;
        console.log(`Being called ${counter} times`)
    }
}

var fn = callCounter()
fn()
fn()

Being called 1 times
Being called 2 times


For closure to come into effect, we don't necessarily need an outer function. An inner function inside an outer scope is enough.

In [26]:
var hits

{
    // Scope defined by the curly braces
    let count = 0
    hits = function(){
        count++
        console.log(count)
    }
}

hits()
hits()
hits()

1
2
3


Close is *variable-oriented* and not *value-oriented*.

In [25]:
var functions = []

for(let i=0; i<5; i++){
    functions[i] = function(){
        return i*i;
    }
}

console.log(functions[1]())
console.log(functions[2]())
console.log(functions[3]())

1
4
9


Whereas if we use `var`, we get

In [1]:
var functions = []

for(var i=0; i<5; i++){
    functions[i] = function(){
        return i*i;
    }
}

console.log(functions[1]())
console.log(functions[2]())
console.log(functions[3]())

25
25
25


The reason for the difference lies in how JavaScript interprets `let` in a for loop. When we use `let` instead of `var`, JavaScript creates a new variable for each iteration.

In [ ]:
for(let i=0; i<5; i++){
    console.log(i)
}

// is equivalent to:
for(var i=0; i<5; i++){
    let j = i;
    console.log(j)
}

It has already been mentioned that `eval` is source of lot of confusion. If we consider the below snippet

In [ ]:
function storeStudentInfo(id,name,grade) { // none of these variables are being used inside the inner function, so should have
                                           // been garbage collected. But no!
    return function getInfo(whichValue){
        //   using `eval(..)` is a bad idea!
        var val = eval(whichValue);
        return val;
    };
}

var info = storeStudentInfo(73,"Suzy",87);

console.log(info("name")); // Suzy
console.log(info("grade"));// 87